<a href="https://colab.research.google.com/github/robimalco/COVID19_analysis/blob/master/CoronaVirusITALY_PROVINCES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DEFINE LIBRARIES**

In [0]:
import pandas as pd
import numpy as np
import io
import requests
from google.colab import files
pd.options.mode.chained_assignment = None
! pip install geopandas

# GEODATA
import os
import folium
from folium import plugins
import geopandas as gpd

# **DOWNLOAD PROVINCE DATA**

In [0]:
url_first_part = 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-province/dpc-covid19-ita-province-'

file_dates = [
              '20200224.csv', '20200225.csv', '20200226.csv', '20200227.csv', '20200228.csv', '20200229.csv',   
              '20200301.csv', '20200302.csv', '20200303.csv', '20200304.csv', '20200305.csv', '20200306.csv',
              '20200307.csv', '20200308.csv', '20200309.csv', '20200310.csv', '20200311.csv', '20200312.csv',
              '20200313.csv', '20200314.csv', '20200315.csv', '20200316.csv', '20200317.csv', '20200318.csv',
              '20200319.csv', '20200320.csv', '20200321.csv', '20200322.csv', '20200323.csv', '20200324.csv',
              '20200325.csv', '20200326.csv', '20200327.csv', '20200328.csv', '20200329.csv', '20200330.csv',
              '20200331.csv', '20200401.csv', '20200402.csv', '20200403.csv', '20200404.csv', '20200405.csv',
              '20200406.csv', '20200407.csv', '20200408.csv', '20200409.csv', '20200410.csv', '20200411.csv',
              '20200412.csv', '20200413.csv', '20200414.csv', '20200415.csv', '20200416.csv', '20200417.csv']

full_data_list = []

for i in range(0, len(file_dates)):
  # print('Processing: ' + file_dates[i])
  url_complete = url_first_part + file_dates[i]
  response = requests.get(url_complete).content
  temp_df = pd.read_csv(io.StringIO(response.decode('utf-8')))
  full_data_list.append(temp_df)

df = pd.concat(full_data_list, ignore_index=True)

# df.to_csv('df.csv')
# files.download('df.csv')

# **DOWNLOAD INHABITANTS PER PROVINCE**

In [0]:
from bs4 import BeautifulSoup
import requests
html = requests.get('https://www.tuttitalia.it/province/popolazione/').text
soup = BeautifulSoup(html)
province_names = soup.select('table.ut')[0].findAll('a')
province_populations = soup.findAll('td', {'class': 'cw'})
df_province_inhabitants = pd.DataFrame(columns=['sigla_provincia', 'inhabitants'])
prov_rows = soup.select('table.ut')[0].findAll('tr')
for prov_row in prov_rows:
  if '<div class="ow">' in str(prov_row):
    df_province_inhabitants = df_province_inhabitants.append({
        'sigla_provincia': prov_row.find('div', {'class': 'ow'}).getText(),
        'inhabitants': int(prov_row.find('td', {'class': 'cw'}).getText().replace('.', ''))
      }, ignore_index=True)

# **DOWNLOAD GEODATA**

In [0]:
# Download Geodata
file_name = 'https://raw.githubusercontent.com/openpolis/geojson-italy/master/geojson/limits_IT_provinces.geojson'
province_geo_data = gpd.read_file(file_name)
province_geo_data['prov_name'] = province_geo_data['prov_name'].str.lower()

# **FIX INCONSISTENCIES BETWEEN PROVINCES' NAMES**

In [0]:
# Change name to specific provinces to match geo spatial names and add useful column
df.loc[df['denominazione_provincia'] == 'Aosta', 'denominazione_provincia'] = "Valle d'Aosta/Vallée d'Aoste"
df.loc[df['denominazione_provincia'] == 'Bolzano', 'denominazione_provincia'] = 'Bolzano/Bozen'
df.loc[df['denominazione_provincia'] == 'Massa Carrara', 'denominazione_provincia'] = 'Massa-Carrara'

# **FILTER, MERGE AND GROUP DATA**

In [0]:
# Create copy of df to avoid not useful re-estraction
province_df = df

province_df = province_df.merge(df_province_inhabitants, on='sigla_provincia')

province_df = province_df[province_df['denominazione_provincia'] != 'In fase di definizione/aggiornamento']

# **ADD NEW INTERESTING COLUMNS**

In [0]:
province_df['inhabitants/totale_casi'] = province_df['inhabitants'].div(province_df['totale_casi'].where(province_df['totale_casi'] != 0, np.nan)).fillna(0).replace([np.inf, -np.inf], 0).astype(int)

# **ANALYZE DATA**

In [0]:
kpi_columns = ['totale_casi', 'inhabitants/totale_casi']

ok_province_df = pd.DataFrame()

province_df['days_from_last_obs'] = (pd.to_datetime(province_df.data.max()) - pd.to_datetime(province_df.data, errors='coerce')).dt.days
temp_province_df = province_df.sort_values(by=['denominazione_regione', 'days_from_last_obs'], ascending=[False, False])

for q in range(0, len(kpi_columns)): 
  temp_province_df['d_' + kpi_columns[q]] = temp_province_df.groupby(['denominazione_provincia'])[kpi_columns[q]].diff().fillna(0).replace([np.inf, -np.inf], 0).astype(int)
  temp_province_df['dp_' + kpi_columns[q]] = temp_province_df.groupby(['denominazione_provincia'])[kpi_columns[q]].pct_change().apply(lambda x: round(x, 2)* 100).fillna(0).replace([np.inf, -np.inf], 0).astype(int)
ok_province_df = ok_province_df.append(temp_province_df)

# **SINGLE PROVINCE RECAP**

In [0]:
COLUMNS_TO_DISPLAY = ['days_from_last_obs', 'data', 'denominazione_provincia', 'totale_casi', 'd_totale_casi', 'dp_totale_casi', 'inhabitants/totale_casi']

RECAP_PROVINCE = 'Varese'

LAST_n_DAYS = 10

ok_province_df_recap = ok_province_df[COLUMNS_TO_DISPLAY]
ok_province_df_recap[(ok_province_df_recap['denominazione_provincia'] == RECAP_PROVINCE) & (ok_province_df_recap['days_from_last_obs'] < LAST_n_DAYS)]

## **MULTI PROVINCE RECAP**

In [0]:
COLUMNS_TO_DISPLAY = ['days_from_last_obs', 'data', 'denominazione_provincia', 'totale_casi', 'd_totale_casi', 'dp_totale_casi', 'inhabitants/totale_casi']

RECAP_PROVINCIES = ['Palermo', 'Messina', 'Catania', 'Agrigento', 'Caltanissetta', 'Enna', 'Trapani', 'Ragusa']

DAYS_AGO = 0

ok_province_df_recap = ok_province_df[COLUMNS_TO_DISPLAY]
ok_province_df_recap[(ok_province_df['days_from_last_obs'] == DAYS_AGO) & (ok_province_df['denominazione_provincia'].isin(RECAP_PROVINCIES))]

# **SHOW PIVOT**

In [0]:
PIVOT_MAIN_KPI = 'd_totale_casi'

PIVOT_PROVINCIES = ['Varese', 'Milano', 'Bergamo', 'Brescia', 'Cremona']

LAST_n_DAYS = 15

ok_province_df_for_pivot = ok_province_df.groupby(['denominazione_provincia', 'days_from_last_obs']).last().reset_index()
ok_province_df_pivot = ok_province_df_for_pivot[ok_province_df_for_pivot['denominazione_provincia'].isin(PIVOT_PROVINCIES)].pivot(index='denominazione_provincia', columns='days_from_last_obs', values=[PIVOT_MAIN_KPI]).fillna(0)
ok_province_df_pivot[ok_province_df_pivot.columns[::-1][len(list(ok_province_df_pivot.columns))-LAST_n_DAYS:]]

# **PLOT GEODATA**

In [0]:
KPI_MAP = 'dp_totale_casi'
DAYS_AGO = 0 # 0 is the last data point

map_italy = folium.Map([41.9028, 12.4964], zoom_start=6)

popups = ok_province_df[(ok_province_df['days_from_last_obs'] == DAYS_AGO)]
popups = popups[['denominazione_provincia', KPI_MAP]]

popups.columns = ['prov_name', KPI_MAP]
popups['prov_name'] = popups['prov_name'].str.lower()

province_geo_data_short = province_geo_data.merge(popups,on="prov_name")

province_geo_data_short = province_geo_data_short[['prov_name', KPI_MAP, 'geometry']]

folium.Choropleth(
    geo_data=province_geo_data_short,
    data=province_geo_data_short,
    columns=['prov_name', KPI_MAP],
    key_on='feature.properties.prov_name',
    fill_color='YlOrRd'
).add_to(map_italy)

import branca.colormap as cm

style_function = lambda x: {
    'fillColor': '#ffffff',
    'color':'#000000', 
    'fillOpacity': 0.1,
    'weight': 0.1}
highlight_function = lambda x: {
    'fillColor': '#000000',
    'color':'#000000',
    'fillOpacity': 0.50,
    'weight': 0.1}
NIL = folium.features.GeoJson(
    province_geo_data_short,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['prov_name', KPI_MAP],
        aliases=['prov_name: ', KPI_MAP + ' : '],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
map_italy.add_child(NIL)
map_italy.keep_in_front(NIL)

map_italy